In [1]:
import numpy as np
import pandas as pd

In [2]:
ratings = pd.read_csv("svd/movielens/ratings.csv")

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [4]:
movies = pd.read_csv("svd/movielens/movies.csv")

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
users = ratings.userId.unique().shape[0]
users

7120

In [15]:
n_movies = movies.movieId.unique().shape[0]
n_movies

27278

In [16]:
Ratings = ratings.pivot_table(values='rating',index='userId',columns='movieId').fillna(0)

In [17]:
Ratings.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,129350,129354,129428,129707,130052,130073,130219,130462,130490,130642
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
from surprise import SVD, Reader, Dataset
from surprise.model_selection import cross_validate

In [19]:
reader = Reader()

In [21]:
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

In [22]:
svd = SVD()

In [23]:
cross_validate(svd, data, measures=['RMSE','MAE'],cv=3,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8473  0.8447  0.8447  0.8456  0.0012  
MAE (testset)     0.6493  0.6474  0.6466  0.6478  0.0011  
Fit time          68.65   72.62   71.18   70.82   1.64    
Test time         6.02    6.23    5.36    5.87    0.37    


{'test_rmse': array([0.84730552, 0.84472098, 0.84474559]),
 'test_mae': array([0.64928256, 0.64737122, 0.64663013]),
 'fit_time': (68.65427947044373, 72.62156295776367, 71.18183875083923),
 'test_time': (6.018893003463745, 6.2313196659088135, 5.355545282363892)}

In [53]:
rating5 = ratings[(ratings['userId'] == 5) & (ratings['rating'] == 5)]
rating5 = rating5.set_index('movieId')
rating5 = rating5.join(movies)['title']
rating5.head(10)

movieId
11                    Dracula: Dead and Loving It (1995)
62     Don't Be a Menace to South Central While Drink...
141                                         Gospa (1995)
150                                Addiction, The (1995)
260                             Ladybird Ladybird (1994)
318    Strawberry and Chocolate (Fresa y chocolate) (...
364                                      Maverick (1994)
368                                 Reality Bites (1994)
377                      When a Man Loves a Woman (1994)
380                                   Bad Company (1995)
Name: title, dtype: object

In [33]:
user5 = movies.copy()

In [34]:
user5.reset_index()

,index,movieId,title,genres
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,2,Jumanji (1995),Adventure|Children|Fantasy
2,2,3,Grumpier Old Men (1995),Comedy|Romance
3,3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...,...
27273,27273,131254,Kein Bund für's Leben (2007),Comedy
27274,27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,27275,131258,The Pirates (2014),Adventure
27276,27276,131260,Rentun Ruusu (2001),(no genres listed)


In [26]:
data1 = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

In [27]:
trainset = data1.build_full_trainset()

In [28]:
svd.fit(trainset)

In [35]:
user5['Estimate'] = user5['movieId'].apply(lambda x: svd.predict(5, x).est)

In [36]:
user5 = user5.sort_values(by = 'Estimate', ascending = False)
user5.head()

,movieId,title,genres,Estimate
1173,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,5.000000
315,318,"Shawshank Redemption, The (1994)",Crime|Drama,5.000000
18528,92259,Intouchables (2011),Comedy|Drama,4.970778
360,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,4.963341
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,4.958371
